Importing data from yfinance

In [80]:
try:
  import yfinance as yf
  import numpy as np
except ModuleNotFoundError:
  %pip install yfinance
  %pip install numpy
  import yfinance as yf

In [81]:
rounds=14
ticker = 'BTC'
fast_period , slow_period = 20, 60
sigma = 40

data = yf.download(ticker, auto_adjust=True, start = '2000-01-01')

[*********************100%%**********************]  1 of 1 completed


In [82]:
# we are performing the operation
# mean() function finds the mean over each window
data[f'SMA_{fast_period}']=data.Close.rolling(fast_period).mean()
data[f'SMA_{slow_period}']=data.Close.rolling(slow_period).mean()
data[f'Intersection_{fast_period}_{slow_period}']=data[f'SMA_{fast_period}']/data[f'SMA_{slow_period}']* 100-100
data

,Open,High,Low,Close,Volume,SMA_20,SMA_60,Intersection_20_60
Date,,,,,,,,
2020-10-01,96.966415,96.966415,96.966415,96.966415,0,NaN,NaN,NaN
2020-10-02,96.966415,96.966415,96.966415,96.966415,0,NaN,NaN,NaN
2020-10-05,96.995490,97.005189,96.961555,96.961555,10725,NaN,NaN,NaN
2020-10-06,96.976093,96.995483,96.976093,96.995483,10080,NaN,NaN,NaN
2020-10-07,96.917921,96.927620,96.888832,96.888832,6350,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2023-10-16,89.059998,89.144997,89.059998,89.144997,343,89.243359,89.758596,-0.574025
2023-10-17,88.690002,88.690002,88.690002,88.690002,144,89.195819,89.728884,-0.594084
2023-10-18,88.494102,88.494102,88.494102,88.494102,3,89.141978,89.697106,-0.618892


In [83]:
#RSI
data['diff']= data.Close.diff()
data['win']= np.where(data['diff']>0,data['diff'],0)
data['loss']= np.where(data['diff']<0,abs(data['diff']),0)
data['EMA_win'] = data.win.ewm(alpha=1/rounds).mean()
data['EMA_loss'] = data.loss.ewm(alpha=1/rounds).mean()
data['RS'] = data.EMA_win / data.EMA_loss
data['RSI'] = 100 - (100 /1+data.RS)
data


,Open,High,Low,Close,Volume,SMA_20,SMA_60,Intersection_20_60,diff,win,loss,EMA_win,EMA_loss,RS,RSI
Date,,,,,,,,,,,,,,,
2020-10-01,96.966415,96.966415,96.966415,96.966415,0,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,NaN,NaN
2020-10-02,96.966415,96.966415,96.966415,96.966415,0,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN
2020-10-05,96.995490,97.005189,96.961555,96.961555,10725,NaN,NaN,NaN,-0.004860,0.000000,0.004860,0.000000,0.001741,0.000000,0.000000
2020-10-06,96.976093,96.995483,96.976093,96.995483,10080,NaN,NaN,NaN,0.033928,0.033928,0.000000,0.009447,0.001257,7.518174,-7.518174
2020-10-07,96.917921,96.927620,96.888832,96.888832,6350,NaN,NaN,NaN,-0.106651,0.000000,0.106651,0.007268,0.025569,0.284229,-0.284229
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-16,89.059998,89.144997,89.059998,89.144997,343,89.243359,89.758596,-0.574025,-0.205002,0.000000,0.205002,0.083573,0.102156,0.818091,-0.818091
2023-10-17,88.690002,88.690002,88.690002,88.690002,144,89.195819,89.728884,-0.594084,-0.454994,0.000000,0.454994,0.077603,0.127359,0.609329,-0.609329
2023-10-18,88.494102,88.494102,88.494102,88.494102,3,89.141978,89.697106,-0.618892,-0.195900,0.000000,0.195900,0.072060,0.132254,0.544860,-0.544860


In [84]:
# volatility
data['var'] = data.Close.pct_change()*100 # The pct_change() method returns a DataFrame with the percentage difference between the values for each row and, by default, the previous row.
data[f'sigma_{sigma}'] = data['var'].rolling(sigma).std()*(sigma**0.5)
data

,Open,High,Low,Close,Volume,SMA_20,SMA_60,Intersection_20_60,diff,win,loss,EMA_win,EMA_loss,RS,RSI,var,sigma_40
Date,,,,,,,,,,,,,,,,,
2020-10-01,96.966415,96.966415,96.966415,96.966415,0,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN
2020-10-02,96.966415,96.966415,96.966415,96.966415,0,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,0.000000,NaN
2020-10-05,96.995490,97.005189,96.961555,96.961555,10725,NaN,NaN,NaN,-0.004860,0.000000,0.004860,0.000000,0.001741,0.000000,0.000000,-0.005012,NaN
2020-10-06,96.976093,96.995483,96.976093,96.995483,10080,NaN,NaN,NaN,0.033928,0.033928,0.000000,0.009447,0.001257,7.518174,-7.518174,0.034991,NaN
2020-10-07,96.917921,96.927620,96.888832,96.888832,6350,NaN,NaN,NaN,-0.106651,0.000000,0.106651,0.007268,0.025569,0.284229,-0.284229,-0.109955,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-16,89.059998,89.144997,89.059998,89.144997,343,89.243359,89.758596,-0.574025,-0.205002,0.000000,0.205002,0.083573,0.102156,0.818091,-0.818091,-0.229437,1.480501
2023-10-17,88.690002,88.690002,88.690002,88.690002,144,89.195819,89.728884,-0.594084,-0.454994,0.000000,0.454994,0.077603,0.127359,0.609329,-0.609329,-0.510398,1.537824
2023-10-18,88.494102,88.494102,88.494102,88.494102,3,89.141978,89.697106,-0.618892,-0.195900,0.000000,0.195900,0.072060,0.132254,0.544860,-0.544860,-0.220882,1.550635


In [85]:
data['fw_10'] = (data.Close.shift(-10) / data.Close.shift() - 1 )*100
data['fw_20'] = (data.Close.shift(-20) / data.Close.shift() - 1 )*100
data

,Open,High,Low,Close,Volume,SMA_20,SMA_60,Intersection_20_60,diff,win,loss,EMA_win,EMA_loss,RS,RSI,var,sigma_40,fw_10,fw_20
Date,,,,,,,,,,,,,,,,,,,
2020-10-01,96.966415,96.966415,96.966415,96.966415,0,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
2020-10-02,96.966415,96.966415,96.966415,96.966415,0,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,0.000000,NaN,-0.075014,-0.300128
2020-10-05,96.995490,97.005189,96.961555,96.961555,10725,NaN,NaN,NaN,-0.004860,0.000000,0.004860,0.000000,0.001741,0.000000,0.000000,-0.005012,NaN,-0.110011,-0.297602
2020-10-06,96.976093,96.995483,96.976093,96.995483,10080,NaN,NaN,NaN,0.033928,0.033928,0.000000,0.009447,0.001257,7.518174,-7.518174,0.034991,NaN,-0.147896,-0.330035
2020-10-07,96.917921,96.927620,96.888832,96.888832,6350,NaN,NaN,NaN,-0.106651,0.000000,0.106651,0.007268,0.025569,0.284229,-0.284229,-0.109955,NaN,-0.205634,-0.055162
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-16,89.059998,89.144997,89.059998,89.144997,343,89.243359,89.758596,-0.574025,-0.205002,0.000000,0.205002,0.083573,0.102156,0.818091,-0.818091,-0.229437,1.480501,NaN,NaN
2023-10-17,88.690002,88.690002,88.690002,88.690002,144,89.195819,89.728884,-0.594084,-0.454994,0.000000,0.454994,0.077603,0.127359,0.609329,-0.609329,-0.510398,1.537824,NaN,NaN
2023-10-18,88.494102,88.494102,88.494102,88.494102,3,89.141978,89.697106,-0.618892,-0.195900,0.000000,0.195900,0.072060,0.132254,0.544860,-0.544860,-0.220882,1.550635,NaN,NaN


In [86]:
# Cleaning cols
cols = ['Close',f'Intersection_{fast_period}_{slow_period}','RSI',f'sigma_{sigma}','var','fw_10','fw_20']
data = data.loc[:,cols].dropna().round(4)
data

,Close,Intersection_20_60,RSI,sigma_40,var,fw_10,fw_20
Date,,,,,,,
2020-12-24,96.9923,0.0681,-1.2875,0.5406,0.0366,-0.3252,-0.0751
2020-12-28,97.0275,0.0707,-1.4231,0.5275,0.0363,-0.3619,-0.1304
2020-12-29,97.0294,0.0706,-1.4311,0.5215,0.0020,-0.3059,-0.2080
2020-12-30,97.0678,0.0790,-1.6026,0.5224,0.0396,-0.3378,-0.2156
2020-12-31,97.1167,0.0909,-1.8377,0.5215,0.0503,-0.2966,-0.2334
...,...,...,...,...,...,...,...
2023-09-18,89.8122,-0.2773,-0.8315,1.3761,0.0027,-0.9479,-0.7403
2023-09-19,89.6408,-0.2514,-0.6952,1.3807,-0.1909,-1.2662,-1.2495
2023-09-20,89.5709,-0.2253,-0.6485,1.3806,-0.0779,-0.7483,-1.2792


In [87]:
# Fixing look-ahead bias
# operate today with the indicators of the past
data.iloc[:,1:-1].shift()
data

,Close,Intersection_20_60,RSI,sigma_40,var,fw_10,fw_20
Date,,,,,,,
2020-12-24,96.9923,0.0681,-1.2875,0.5406,0.0366,-0.3252,-0.0751
2020-12-28,97.0275,0.0707,-1.4231,0.5275,0.0363,-0.3619,-0.1304
2020-12-29,97.0294,0.0706,-1.4311,0.5215,0.0020,-0.3059,-0.2080
2020-12-30,97.0678,0.0790,-1.6026,0.5224,0.0396,-0.3378,-0.2156
2020-12-31,97.1167,0.0909,-1.8377,0.5215,0.0503,-0.2966,-0.2334
...,...,...,...,...,...,...,...
2023-09-18,89.8122,-0.2773,-0.8315,1.3761,0.0027,-0.9479,-0.7403
2023-09-19,89.6408,-0.2514,-0.6952,1.3807,-0.1909,-1.2662,-1.2495
2023-09-20,89.5709,-0.2253,-0.6485,1.3806,-0.0779,-0.7483,-1.2792


In [90]:
#R2 vs features
features = [f'Intersection_{fast_period}_{slow_period}','RSI',f'sigma_{sigma}'] 
data.corr().loc[['fw_10','fw_20'],features].round(2)


,Intersection_20_60,RSI,sigma_40
fw_10,-0.06,-0.07,0.07
fw_20,-0.12,-0.04,0.08
